# This is a data scraping exercise

## import

In [2]:
pip install requests

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install selenium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install webdriver_manager

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import InvalidSessionIdException, WebDriverException, TimeoutException
import time
import pandas as pd
import logging
import random
import os

## Main Frame

In [6]:
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Function to initialize WebDriver
def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")  # Run in visible mode for manual CAPTCHA solving
    # options.add_argument("--headless=new")  # Uncomment for headless, but CAPTCHA may block
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.7339.185 Safari/537.36")
    try:
        service = Service(ChromeDriverManager().install())
        return webdriver.Chrome(service=service, options=options)
    except Exception as e:
        logging.error(f"Failed to initialize WebDriver: {e}")
        raise

In [7]:
try:
    driver = initialize_driver()
except Exception as e:
    logging.error("Could not start WebDriver. Ensure Chrome is installed and up to date.")
    exit(1)

base_url = "https://batdongsan.com.vn/nha-dat-ban/p{}"
all_data = []

2025-09-18 17:47:13,716 - INFO - ====== WebDriver manager ======
2025-09-18 17:47:16,996 - INFO - Get LATEST chromedriver version for google-chrome
2025-09-18 17:47:17,308 - INFO - Get LATEST chromedriver version for google-chrome
2025-09-18 17:47:17,456 - INFO - Driver [C:\Users\tuanm\.wdm\drivers\chromedriver\win64\140.0.7339.185\chromedriver-win32/chromedriver.exe] found in cache


In [8]:
# Loop through pages
max_pages = 3  # Test with small range
for page in range(1, max_pages + 1):
    if page == 1:
        url = "https://batdongsan.com.vn/nha-dat-ban"
    else:
        url = base_url.format(page)
    
    logging.info(f"Scraping page {page}: {url}")
    
    # Load page with retries
    retries = 3
    for attempt in range(retries):
        try:
            driver.get(url)
            # Wait for page to be fully loaded
            WebDriverWait(driver, 10).until(lambda d: d.execute_script("return document.readyState") == "complete")
            break
        except InvalidSessionIdException:
            logging.warning("Invalid session ID detected. Reinitializing WebDriver.")
            driver.quit()
            driver = initialize_driver()
            time.sleep(2)
        except WebDriverException as e:
            logging.error(f"WebDriver error on attempt {attempt + 1}: {e}")
            if attempt == retries - 1:
                logging.error("Max retries reached. Skipping page.")
                break
            time.sleep(2)
    
    # Scroll to load listings
    for _ in range(5):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
    
    time.sleep(3)  # Allow lazy load
    
    try:
        wait = WebDriverWait(driver, 30)  # Increased timeout
        listings = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.js__card.js__card-full-web")))
    except TimeoutException:
        logging.error(f"No listings found on page {page}. Checking for CAPTCHA or page content.")
        # Log page HTML for debugging
        html = driver.page_source
        with open(f"page_{page}_error.html", "w", encoding="utf-8") as f:
            f.write(html)
        logging.info(f"Saved page HTML to page_{page}_error.html")
        # Check for CAPTCHA
        if "captcha" in html.lower() or "cloudflare" in html.lower():
            logging.error("Cloudflare CAPTCHA detected. Please solve manually or use a CAPTCHA service.")
            input("Press Enter after solving the CAPTCHA manually...")
            # Retry loading listings after manual intervention
            try:
                listings = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.js__card.js__card-full-web")))
            except TimeoutException:
                logging.error("Retry failed after manual CAPTCHA solving.")
                break
        # Try alternative selector
        else:
            try:
                listings = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.re__card")))
                logging.info("Found listings with alternative selector 'div.re__card'.")
            except:
                logging.error("Alternative selector failed. Stopping.")
                break
    
    logging.info(f"Found {len(listings)} listings on page {page}.")
    
    for listing in listings:
        try:
            url_elem = listing.find_element(By.CSS_SELECTOR, "a.js__product-link-for-product-id")
            url = url_elem.get_attribute("href") if url_elem else None
        except:
            url = None
        
        try:
            title_elem = listing.find_element(By.CLASS_NAME, "re__card-info")
            title = title_elem.get_attribute("title") if title_elem else None
        except:
            title = None
        
        try:
            image_elem = listing.find_element(By.CSS_SELECTOR, "div.re__card-image img")
            image = image_elem.get_attribute("src") if image_elem else None
        except:
            image = None
        
        try:
            price = listing.find_element(By.CSS_SELECTOR, "span.re__card-config-price.js__card-config-item").text
        except:
            price = None
        
        try:
            area = listing.find_element(By.CSS_SELECTOR, "span.re__card-config-area.js__card-config-item").text
        except:
            area = None
        
        try:
            price_per_m2 = listing.find_element(By.CSS_SELECTOR, "span.re__card-config-price_per_m2.js__card-config-item").text
        except:
            price_per_m2 = None
        
        try:
            bedrooms_elem = listing.find_element(By.CSS_SELECTOR, "span.re__card-config-bedroom.js__card-config-item span")
            bedrooms = bedrooms_elem.text if bedrooms_elem else None
        except:
            bedrooms = None
        
        try:
            toilets_elem = listing.find_element(By.CSS_SELECTOR, "span.re__card-config-toilet.js__card-config-item span")
            toilets = toilets_elem.text if toilets_elem else None
        except:
            toilets = None
        
        try:
            location = listing.find_element(By.CSS_SELECTOR, "div.re__card-location span:nth-of-type(2)").text
        except:
            location = None
        
        all_data.append({
            "title": title,
            "url": url,
            "image": image,
            "price": price,
            "area": area,
            "price_per_m2": price_per_m2,
            "bedrooms": bedrooms,
            "toilets": toilets,
            "location": location
        })
    
    # Random delay to avoid rate limiting
    time.sleep(random.uniform(10, 15))  # Increased delay to 10-15 seconds

2025-09-18 17:51:03,076 - INFO - Scraping page 1: https://batdongsan.com.vn/nha-dat-ban
2025-09-18 17:51:03,090 - WARNING - Invalid session ID detected. Reinitializing WebDriver.
2025-09-18 17:51:05,253 - INFO - ====== WebDriver manager ======
2025-09-18 17:51:07,110 - INFO - Get LATEST chromedriver version for google-chrome
2025-09-18 17:51:07,259 - INFO - Get LATEST chromedriver version for google-chrome
2025-09-18 17:51:07,394 - INFO - Driver [C:\Users\tuanm\.wdm\drivers\chromedriver\win64\140.0.7339.185\chromedriver-win32/chromedriver.exe] found in cache
2025-09-18 17:52:00,192 - ERROR - No listings found on page 1. Checking for CAPTCHA or page content.
2025-09-18 17:52:00,215 - INFO - Saved page HTML to page_1_error.html
2025-09-18 17:52:00,218 - ERROR - Cloudflare CAPTCHA detected. Please solve manually or use a CAPTCHA service.
2025-09-18 17:52:35,444 - ERROR - Retry failed after manual CAPTCHA solving.


In [9]:
# Save results to CSV
try:
    df = pd.DataFrame(all_data)
    df.to_csv("batdongsan_listings.csv", index=False, encoding="utf-8-sig")
    logging.info(f"Scraped {len(df)} listings across {page-1} pages. Saved to batdongsan_listings.csv")
except Exception as e:
    logging.error(f"Error saving CSV: {e}")

2025-09-18 17:52:51,480 - INFO - Scraped 0 listings across 0 pages. Saved to batdongsan_listings.csv


In [10]:
# Clean up
try:
    driver.quit()
    logging.info("WebDriver closed successfully.")
except:
    logging.warning("WebDriver was already closed or could not be closed cleanly.")

2025-09-18 17:52:55,991 - INFO - WebDriver closed successfully.


## HTML ELEMENTS

In [ ]:
### next page
''' 
<div class="js__card js__card-full-web
     pr-container re__card-full  re__vip-diamond" prav="https://file4.batdongsan.com.vn/crop/200x140/2025/09/18/20250918153019-3630_wm.jpg" uid="3300690" prid="44050053" vtp="vip-diamond" ipos="1" pgno="3" clo="srpg" "="" card-tracking-id="navigate-srp-ldp" card-tracking-label="ipos=1,pgno=3,prid=44050053,verified=0,vipType=V0,isMapSearch=0,showMap=0">

    <a class="js__product-link-for-product-id" data-product-id="44050053" href="/ban-nha-biet-thu-lien-ke-duong-dh20-xa-long-hung-prj-vinhomes-ocean-park-2/can-goc-bt-sieu-hiem-3-mat-tien-view-trieu-do-sat-cong-vien-10-toa-cc-va-san-golf-tai-ocp2-pr44050053" title="Căn góc BT siêu hiếm 3 mặt tiền, view triệu đô, sát công viên, 10 tòa CC và sân golf tại OCP2">
        <div class="re__card-image ">



        <div class="re__img-parent">
            <div class="re__img-child">
                <div>
                    <img error-image-class="error-photo-md" src="https://file4.batdongsan.com.vn/crop/562x284/2025/09/18/20250918153019-3630_wm.jpg" fetchpriority="high" alt="Ảnh đại diện" data-img="https://file4.batdongsan.com.vn/crop/562x284/2025/09/18/20250918153019-3630_wm.jpg">
                </div>
            </div>
        </div>
        <div class="re__img-parent re__img-parent1">
            <div class="re__img-child re__img-child1">
                <div>
                    <img error-image-class="error-photo-md" src="https://file4.batdongsan.com.vn/crop/283x141/2025/09/18/20250918153019-9c35_wm.jpg" alt="Ảnh đại diện" data-img="https://file4.batdongsan.com.vn/crop/283x141/2025/09/18/20250918153019-9c35_wm.jpg">
                </div>
            </div>
        </div>
        <div class="re__img-parent re__img-parent2">
            <div class="re__img-child re__img-child2">
                <div>
                    <img error-image-class="error-photo-xs" src="https://file4.batdongsan.com.vn/crop/140x140/2025/09/18/20250918153019-d7d2_wm.jpg" alt="Ảnh đại diện" data-img="https://file4.batdongsan.com.vn/crop/140x140/2025/09/18/20250918153019-d7d2_wm.jpg">
                </div>
            </div>
        </div>
        <div class="re__img-parent re__img-parent2">
            <div class="re__img-child re__img-child2">
                <div>
                    <img error-image-class="error-photo-xs" src="https://file4.batdongsan.com.vn/crop/140x140/2025/09/18/20250918153019-bb01_wm.jpg" alt="Ảnh đại diện" data-img="https://file4.batdongsan.com.vn/crop/140x140/2025/09/18/20250918153019-bb01_wm.jpg">
                </div>
            </div>
        </div>

            <div class="re__clear"></div>
            <div class="card-label-img label--v1"></div>

            <div class="re__card-image-feature">
                        <i class="re__icon-image"></i>
                    <span>4</span>
            </div>
        </div>
        <div class="re__card-info" title="Căn góc BT siêu hiếm 3 mặt tiền, view triệu đô, sát công viên, 10 tòa CC và sân golf tại OCP2">
            <div class="re__card-info-content">
                <h3 class="re__card-title">
                    <span class="pr-title js__card-title" product-title="">
                        Căn góc BT siêu hiếm 3 mặt tiền, view triệu đô, sát công viên, 10 tòa CC và sân golf tại OCP2
                    </span>
                </h3>
                <div>
                    <div class="re__card-config js__card-config">
                        <span class="re__card-config-price js__card-config-item">81,35 tỷ</span>
                            <span class="re__card-config-dot js__card-config-item">·</span>
                            <span class="re__card-config-area js__card-config-item">378 m²</span>
                            <span class="re__card-config-dot js__card-config-item">·</span>
                            <span class="re__card-config-price_per_m2 js__card-config-item">215,2 tr/m²</span>

                    </div>
                    <div class="re__card-location">
                            <span class="re__card-config-dot">·</span>
                        <span>Văn Giang, Hưng Yên</span>
                    </div>
                    <div class="re__clear"></div>
                </div>
                    <div class="re__card-description js__card-description">
                        Căn góc biệt thự 3 mặt tiền cực hiếm tại VOCP2.Ngay đường 20m, sát công viên mát xanh.Diện tích rộng 378m², mặt tiền thoáng 30m.Vỉa hè rộng, đỗ xe thoải mái.Đối diện cụm 10 tòa chung cư đông đúc.Vị trí tuyệt vời để ở sang kinh doanh thuận buôn bán sầm uất. Anh chị quan tâm liên hệ em Châu ạ <span class="hidden-mobile " raw="d6677a396f82a87e662b6cac2522f4d4">0966 659 ***</span>.
                    </div>
            </div>
            <div class="re__card-contact">
                <div class="re__card-published-info" title="">
                            <div class="entry-agent-profile js__entry-professional-agent re__normal-account" data-bds-pa-link="https://guru.batdongsan.com.vn/pa/nguyenthiminhchau?productType=38&amp;cateId=325&amp;projectId=5431&amp;cityCode=HY&amp;districtId=339" tracking-id="navigate-pa-detail-from-srp" tracking-label="loc=Sale-Listing,agentId=3300690,prid=44050053,isPa=1">
                                <div class="entry-agent-avatar" data-bds-tooltip="Đây là môi giới chuyên nghiệp đã được xác thực thông tin cá nhân. Có thể tư vấn và hỗ trợ giao dịch nhanh chóng." data-bds-tooltip-width="232" data-bds-tooltip-position="right" data-bds-custom-css-class="pro-agent-srp-card">
                                        <img src="https://file4.batdongsan.com.vn/resize/200x200/2025/05/14/20250514114509-c7e6.jpg">
                                    <div class="re__card-listing-pro-agent-badge"></div>

                                </div>
                                <div class="entry-agent-infor">
                                    <div class="agent-name agent-item" data-bds-tooltip="Đây là môi giới chuyên nghiệp đã được xác thực thông tin cá nhân. Có thể tư vấn và hỗ trợ giao dịch nhanh chóng." data-bds-tooltip-width="232" data-bds-tooltip-position="right" data-bds-custom-css-class="pro-agent-srp-card">
                                        &lrm;Nguyễn Thị Minh Châu
                                    </div>
                                    <div class="agent-listing-time agent-item">
                                                <span class="re__card-published-info-published-at" aria-label="18/09/2025" data-microtip-position="right" role="tooltip">
            Đăng hôm nay
        </span>

                                    </div>
                                </div>
                            </div>
                </div>
                <div class="re__card-contact-button">
                        <span class="re__btn re__btn-cyan-solid--sm re__btn-icon-left--sm js__card-phone-btn" prid="44050053" uid="3300690" raw="d6677a396f82a87e662b6cac2522f4d4" data-kyc-name="Nguyễn Thị Minh Châu" data-kyc-avatar="" kyc-tracking-id="lead-phone-srp" tracking-label="loc=SRP For Sale,prid=44050053">
                            <i class="re__icon-phone-call--sm"></i>
                            <span>0966 659 *** · Hiện số</span>
                        </span>
                        <span class="js__marking-product__KYC re__btn re__btn-se-border--sm re__btn-icon--sm" data-marked-product-info-as-json="{&quot;documentId&quot;:&quot;44050053&quot;,&quot;productId&quot;:44050053,&quot;avatarinfowap&quot;:&quot;https://file4.batdongsan.com.vn/crop/200x140/2025/09/18/20250918153019-3630_wm.jpg&quot;,&quot;title&quot;:&quot;Căn góc BT siêu hiếm 3 mặt tiền, view triệu đô, sát công viên, 10 tòa CC và sân golf tại OCP2&quot;,&quot;url&quot;:&quot;/ban-nha-biet-thu-lien-ke-duong-dh20-xa-long-hung-prj-vinhomes-ocean-park-2/can-goc-bt-sieu-hiem-3-mat-tien-view-trieu-do-sat-cong-vien-10-toa-cc-va-san-golf-tai-ocp2-pr44050053&quot;}" aria-label="Bấm để lưu tin" data-microtip-position="bottom" role="tooltip" kyc-tracking-id="save-listing" tracking-label="loc=SRP For Sale,status=save" title="" data-change-marking="1">
                            <i class="re__icon-heart--sm"></i>
                            <i class="re__icon-heart-pressed--sm"></i>
                        </span>
                </div>
                <div class="re__clear"></div>
            </div>
        </div>
        <div class="re__clear"></div>
    </a>
    <!-- Because div can contains buttons of links. Show when click they can send the going listing tracking -->
    <!-- So we use anorder way to send going listing tracking -->
    <div class="js__tracking-div-44050053" tracking-id="navigate-srp-ldp" tracking-label="ipos=1,pgno=3,prid=44050053,verified=0,vipType=V0,isMapSearch=0,showMap=0"></div>
    <script type="text/javascript">
        if (!window.CallTrackingWhenGoToLdpPage) {
            window.CallTrackingWhenGoToLdpPage = function (event, container, listingId) {
                var containsTarget = false;
                var trackingQueries = ['[tracking-id]', '[lead-tracking-id]', '.js__entry-professional-agent'];
                for (var i = 0; i < trackingQueries.length; i++) {
                    var query = trackingQueries[i];
                    var elements = container.querySelectorAll(query);
                    for (var j = 0; j < elements.length; j++) {
                        var element = elements[j];
                        if (element.contains(event.target)) {
                            containsTarget = true;
                            break;
                        }
                    }
                    if (containsTarget) {
                        break;
                    }
                }

                if (!containsTarget) {
                    document.querySelector(`.js__tracking-div-${listingId}`).dispatchEvent(new Event('click'))
                }
            };
        }
    </script>
</div>
'''

''

In [ ]:
### element 2 
### div.js__card.js__card-full-web
### pr-container re__card-full  re__vip-diamond
'''
<div class="js__card js__card-full-web
     pr-container re__card-full  re__vip-diamond" prav="https://file4.batdongsan.com.vn/crop/200x140/2025/03/16/20250316100720-773f_wm.jpg" uid="4730514" prid="44051322" vtp="vip-diamond" ipos="3" pgno="1" clo="srpg" "="" card-tracking-id="navigate-srp-ldp" card-tracking-label="ipos=3,pgno=1,prid=44051322,verified=0,vipType=V0,isMapSearch=0,showMap=0">

    <a class="js__product-link-for-product-id" data-product-id="44051322" href="/ban-dat-phuong-tien-cat/sieu-pham-mat-ho-view-san-golf-tai-viet-tri-pr44051322" title="SIÊU PHẨM ĐẤT MẶT HỒ  VIEW SÂN GOLF TẠI VIỆT TRÌ">
        <div class="re__card-image ">



        <div class="re__img-parent">
            <div class="re__img-child">
                <div>
                    <img error-image-class="error-photo-md" class=" lazyloaded" data-src="https://file4.batdongsan.com.vn/crop/562x284/2025/03/16/20250316100720-773f_wm.jpg" alt="Ảnh đại diện" data-img="https://file4.batdongsan.com.vn/crop/562x284/2025/03/16/20250316100720-773f_wm.jpg" src="https://file4.batdongsan.com.vn/crop/562x284/2025/03/16/20250316100720-773f_wm.jpg">
                </div>
            </div>
        </div>
        <div class="re__img-parent re__img-parent1">
            <div class="re__img-child re__img-child1">
                <div>
                    <img error-image-class="error-photo-md" class=" lazyloaded" data-src="https://file4.batdongsan.com.vn/crop/283x141/2025/03/16/20250316100719-f985_wm.jpg" alt="Ảnh đại diện" data-img="https://file4.batdongsan.com.vn/crop/283x141/2025/03/16/20250316100719-f985_wm.jpg" src="https://file4.batdongsan.com.vn/crop/283x141/2025/03/16/20250316100719-f985_wm.jpg">
                </div>
            </div>
        </div>
        <div class="re__img-parent re__img-parent2">
            <div class="re__img-child re__img-child2">
                <div>
                    <img error-image-class="error-photo-xs" class=" lazyloaded" data-src="https://file4.batdongsan.com.vn/crop/140x140/2025/03/16/20250316100719-8f02_wm.jpg" alt="Ảnh đại diện" data-img="https://file4.batdongsan.com.vn/crop/140x140/2025/03/16/20250316100719-8f02_wm.jpg" src="https://file4.batdongsan.com.vn/crop/140x140/2025/03/16/20250316100719-8f02_wm.jpg">
                </div>
            </div>
        </div>
        <div class="re__img-parent re__img-parent2">
            <div class="re__img-child re__img-child2">
                <div>
                    <img error-image-class="error-photo-xs" class=" lazyloaded" data-src="https://file4.batdongsan.com.vn/crop/140x140/2025/03/16/20250316100719-1a38_wm.jpg" alt="Ảnh đại diện" data-img="https://file4.batdongsan.com.vn/crop/140x140/2025/03/16/20250316100719-1a38_wm.jpg" src="https://file4.batdongsan.com.vn/crop/140x140/2025/03/16/20250316100719-1a38_wm.jpg">
                </div>
            </div>
        </div>

            <div class="re__clear"></div>
            <div class="card-label-img label--v1"></div>

            <div class="re__card-image-feature">
                        <i class="re__icon-image"></i>
                    <span>8</span>
            </div>
        </div>
        <div class="re__card-info" title="SIÊU PHẨM ĐẤT MẶT HỒ  VIEW SÂN GOLF TẠI VIỆT TRÌ">
            <div class="re__card-info-content">
                <h3 class="re__card-title">
                    <span class="pr-title js__card-title" product-title="">
                        SIÊU PHẨM ĐẤT MẶT HỒ  VIEW SÂN GOLF TẠI VIỆT TRÌ
                    </span>
                </h3>
                <div>
                    <div class="re__card-config js__card-config">
                        <span class="re__card-config-price js__card-config-item">13,88 tỷ</span>
                            <span class="re__card-config-dot js__card-config-item">·</span>
                            <span class="re__card-config-area js__card-config-item">111 m²</span>
                            <span class="re__card-config-dot js__card-config-item">·</span>
                            <span class="re__card-config-price_per_m2 js__card-config-item">125 tr/m²</span>

                    </div>
                    <div class="re__card-location">
                            <span class="re__card-config-dot">·</span>
                        <span>Việt Trì, Phú Thọ</span>
                    </div>
                    <div class="re__clear"></div>
                </div>
                    <div class="re__card-description js__card-description">
                        Diện tích: 111m² (MT 6m  dài 18.5m, vuông vắn, pháp lý chuẩn) Giá chỉ: 125 triệu/m²  cơ hội đầu tư siêu lời Hướng: Tây Nam  hợp phong thủy, đón tài lộc Vị trí kim cương  tiện ích đẳng cấp: View sân Golf cao cấp &amp; Mặt hồ Công viên Văn Lang siêu rộng Gần công viên xanh, sau Nhà thi đấu Tỉnh Trung tâm sôi động nhất Việt Trì  phố lễ hội pháo hoa, nhà hàng, café, giải trí Hạ tầng đồng bộ, giao thông thuận tiện Liên hệ ngay: <span class="hidden-mobile " raw="6e9e5e6e6a434b667c866478d8e678f2">0983 866 ***</span> (Hiếu)
                    </div>
            </div>
            <div class="re__card-contact">
                <div class="re__card-published-info" title="">
                            <div class="entry-agent-profile js__entry-professional-agent re__normal-account" data-bds-pa-link="https://guru.batdongsan.com.vn/pa/c2f67bae1752c106?productType=38&amp;cateId=283&amp;cityCode=PT&amp;districtId=485&amp;wardId=3538" tracking-id="navigate-pa-detail-from-srp" tracking-label="loc=Sale-Listing,agentId=4730514,prid=44051322,isPa=0">
                                <div class="entry-agent-avatar">
                                        <div class="re__card-published-info-agent-profile-avatar-no-photo">H</div>
                                </div>
                                <div class="entry-agent-infor">
                                    <div class="agent-name agent-item">
                                        &lrm;Anh Hiếu
                                    </div>
                                    <div class="agent-listing-time agent-item">
                                                <span class="re__card-published-info-published-at" aria-label="18/09/2025" data-microtip-position="right" role="tooltip">
            Đăng hôm nay
        </span>

                                    </div>
                                </div>
                            </div>
                </div>
                <div class="re__card-contact-button">
                        <span class="re__btn re__btn-cyan-solid--sm re__btn-icon-left--sm js__card-phone-btn" prid="44051322" uid="4730514" raw="6e9e5e6e6a434b667c866478d8e678f2" data-kyc-name="Anh Hiếu" data-kyc-avatar="" kyc-tracking-id="lead-phone-srp" tracking-label="loc=SRP For Sale,prid=44051322">
                            <i class="re__icon-phone-call--sm"></i>
                            <span>0983 866 *** · Hiện số</span>
                        </span>
                        <span class="js__marking-product__KYC re__btn re__btn-se-border--sm re__btn-icon--sm" data-marked-product-info-as-json="{&quot;documentId&quot;:&quot;44051322&quot;,&quot;productId&quot;:44051322,&quot;avatarinfowap&quot;:&quot;https://file4.batdongsan.com.vn/crop/200x140/2025/03/16/20250316100720-773f_wm.jpg&quot;,&quot;title&quot;:&quot;SIÊU PHẨM ĐẤT MẶT HỒ  VIEW SÂN GOLF TẠI VIỆT TRÌ&quot;,&quot;url&quot;:&quot;/ban-dat-phuong-tien-cat/sieu-pham-mat-ho-view-san-golf-tai-viet-tri-pr44051322&quot;}" aria-label="Bấm để lưu tin" data-microtip-position="bottom" role="tooltip" kyc-tracking-id="save-listing" tracking-label="loc=SRP For Sale,status=save" title="" data-change-marking="1">
                            <i class="re__icon-heart--sm"></i>
                            <i class="re__icon-heart-pressed--sm"></i>
                        </span>
                </div>
                <div class="re__clear"></div>
            </div>
        </div>
        <div class="re__clear"></div>
    </a>
    <!-- Because div can contains buttons of links. Show when click they can send the going listing tracking -->
    <!-- So we use anorder way to send going listing tracking -->
    <div class="js__tracking-div-44051322" tracking-id="navigate-srp-ldp" tracking-label="ipos=3,pgno=1,prid=44051322,verified=0,vipType=V0,isMapSearch=0,showMap=0"></div>
    <script type="text/javascript">
        if (!window.CallTrackingWhenGoToLdpPage) {
            window.CallTrackingWhenGoToLdpPage = function (event, container, listingId) {
                var containsTarget = false;
                var trackingQueries = ['[tracking-id]', '[lead-tracking-id]', '.js__entry-professional-agent'];
                for (var i = 0; i < trackingQueries.length; i++) {
                    var query = trackingQueries[i];
                    var elements = container.querySelectorAll(query);
                    for (var j = 0; j < elements.length; j++) {
                        var element = elements[j];
                        if (element.contains(event.target)) {
                            containsTarget = true;
                            break;
                        }
                    }
                    if (containsTarget) {
                        break;
                    }
                }

                if (!containsTarget) {
                    document.querySelector(`.js__tracking-div-${listingId}`).dispatchEvent(new Event('click'))
                }
            };
        }
    </script>
</div>
 
'''

In [ ]:
### elemetnt 1
###pr-container re__card-full re__card-full-label-verified re__vip-diamond
'''
<div class="js__card js__card-full-web
     pr-container re__card-full re__card-full-label-verified re__vip-diamond" prav="https://file4.batdongsan.com.vn/crop/200x140/2025/09/06/20250906104356-ea48_wm.jpg" uid="2337610" prid="43819740" vtp="vip-diamond" ipos="1" pgno="2" clo="srpg" "="" card-tracking-id="navigate-srp-ldp" card-tracking-label="ipos=1,pgno=2,prid=43819740,verified=1,vipType=V0,isMapSearch=0,showMap=0">

    <a class="js__product-link-for-product-id" data-product-id="43819740" href="/ban-can-ho-chung-cu-pho-ton-duc-thang-phuong-ben-nghe-prj-vinhomes-golden-river-ba-son/don-binh-minh-tren-song-sai-gon-moi-ngay-tai-goc-3pn-118m2-dep-nhat-aqua-2-son-pr43819740" title="">
        <div class="re__card-image ">



        <div class="re__img-parent">
            <div class="re__img-child">
                <div>
                    <img error-image-class="error-photo-md" src="https://file4.batdongsan.com.vn/crop/562x284/2025/09/06/20250906104356-ea48_wm.jpg" fetchpriority="high" alt="Ảnh đại diện" data-img="https://file4.batdongsan.com.vn/crop/562x284/2025/09/06/20250906104356-ea48_wm.jpg">
                </div>
            </div>
        </div>
        <div class="re__img-parent re__img-parent1">
            <div class="re__img-child re__img-child1">
                <div>
                    <img error-image-class="error-photo-md" src="https://file4.batdongsan.com.vn/crop/283x141/2025/09/16/20250916105817-72a2_wm.jpg" alt="Ảnh đại diện" data-img="https://file4.batdongsan.com.vn/crop/283x141/2025/09/16/20250916105817-72a2_wm.jpg">
                </div>
            </div>
        </div>
        <div class="re__img-parent re__img-parent2">
            <div class="re__img-child re__img-child2">
                <div>
                    <img error-image-class="error-photo-xs" src="https://file4.batdongsan.com.vn/crop/140x140/2025/09/16/20250916105916-5a1a_wm.jpg" alt="Ảnh đại diện" data-img="https://file4.batdongsan.com.vn/crop/140x140/2025/09/16/20250916105916-5a1a_wm.jpg">
                </div>
            </div>
        </div>
        <div class="re__img-parent re__img-parent2">
            <div class="re__img-child re__img-child2">
                <div>
                    <img error-image-class="error-photo-xs" src="https://file4.batdongsan.com.vn/crop/140x140/2025/09/16/20250916105814-1a75_wm.jpg" alt="Ảnh đại diện" data-img="https://file4.batdongsan.com.vn/crop/140x140/2025/09/16/20250916105814-1a75_wm.jpg">
                </div>
            </div>
        </div>

            <div class="re__clear"></div>
            <div class="card-label-img label--v1"></div>

            <div class="re__card-image-feature">
                        <i class="re__icon-image"></i>
                    <span>22</span>
            </div>
        </div>
        <div class="re__card-info" title="Bán gấp căn hộ góc 3PN Vinhomes Ba Son, 118.5m2, tầng cao, view sông trực diện, giá tốt nhất 29 tỷ">
            <div class="re__card-info-content">
                <h3 class="re__card-title">
                    <span class="pr-title js__card-title" product-title="">
###title                             <span class="re__card-image-verified " data-bds-tooltip-width="220" data-bds-tooltip-position="top" data-bds-tooltip="Tin đăng đã được xác thực thông qua việc kiểm tra sổ đỏ và hình ảnh cung cấp bởi người đăng tin. &lt;a style=&quot;display: flex; justify-content: center; text-align: center; text-decoration: underline; font-size: 13px&quot; class=&quot;js__verified-info-click&quot; href=&quot;javascript:void(0);&quot; onclick=&quot;window.FrontEnd_Product_VerifiedListing.ShowVerifiedListingPopup();&quot; tracking-id=&quot;verified-more-infor-click&quot; tracking-label=&quot;loc=text&quot;&gt; Tìm hiểu thêm&lt;/span&gt;"></span>
                        Bán gấp căn hộ góc 3PN Vinhomes Ba Son, 118.5m2, tầng cao, view sông trực diện, giá tốt nhất 29 tỷ
                    </span>
                </h3>
                <div>
                    <div class="re__card-config js__card-config">
###price                        <span class="re__card-config-price js__card-config-item">29 tỷ</span>
                            <span class="re__card-config-dot js__card-config-item">·</span>
###area                            <span class="re__card-config-area js__card-config-item">118,5 m²</span>
                            <span class="re__card-config-dot js__card-config-item">·</span>
###price/m^2                            <span class="re__card-config-price_per_m2 js__card-config-item">244,73 tr/m²</span>

                            <span class="re__card-config-dot js__card-config-item">·</span>
###bedroom                            <span class="re__card-config-bedroom js__card-config-item" aria-label="3 Phòng ngủ" data-microtip-position="bottom" role="tooltip"><span>3</span><i class="re__icon-bedroom--sm"></i></span>
                            <span class="re__card-config-dot js__card-config-item">·</span>
                            <span class="re__card-config-toilet js__card-config-item" aria-label="2 WC" data-microtip-position="bottom" role="tooltip"><span>2</span><i class="re__icon-bath--sm"></i></span>
                    </div>
#location                    <div class="re__card-location">
                            <span class="re__card-config-dot">·</span>
                        <span>Quận 1, Hồ Chí Minh</span>
                    </div>
                    <div class="re__clear"></div>
                </div>
                    <div class="re__card-description js__card-description">
                        Cơ hội tốt nhất tháng 9! Chủ nhà thiện chí cần bán gấp, giá đã giảm 1 tỷ so với giá thị trường. Sở hữu ngay một trong những căn hộ có tầm nhìn đẹp nhất dự án với mức giá không thể tốt hơn.Thông tin chi tiết: Dự án: Vinhomes Golden River, Ba Son, Quận 1. Vị trí: Tòa Aqua 2, tầng cao. Diện tích tim tường: 118.5 m². Thiết kế: 3 phòng ngủ - 2 WC (căn góc 2 mặt thoáng). Tầm nhìn: Trực diện sông Sài Gòn và Bitexco, không bị che chắn. Tình trạng: Sổ hồn...
                    </div>
            </div>
            <div class="re__card-contact">
                <div class="re__card-published-info" title="">
                            <div class="entry-agent-profile js__entry-professional-agent re__normal-account" data-bds-pa-link="https://guru.batdongsan.com.vn/pa/03f7ce41e4f2fcb6?productType=38&amp;cateId=324&amp;projectId=2393&amp;cityCode=SG&amp;districtId=53" tracking-id="navigate-pa-detail-from-srp" tracking-label="loc=Sale-Listing,agentId=2337610,prid=43819740,isPa=0">
                                <div class="entry-agent-avatar">
                                        <img src="https://file4.batdongsan.com.vn/resize/200x200/2023/05/15/20230515131756-0f7e.jpg">
                                </div>
                                <div class="entry-agent-infor">
                                    <div class="agent-name agent-item">
                                        &lrm;Modan Home
                                    </div>
                                    <div class="agent-listing-time agent-item">
                                                <span class="re__card-published-info-published-at" aria-label="18/09/2025" data-microtip-position="right" role="tooltip">
            Đăng hôm nay
        </span>

                                    </div>
                                </div>
                            </div>
                </div>
                <div class="re__card-contact-button">
                        <span class="re__btn re__btn-cyan-solid--sm re__btn-icon-left--sm js__card-phone-btn" prid="43819740" uid="2337610" raw="f3054966e884a7dc736202beb3068db5" data-kyc-name="Modan Home" data-kyc-avatar="" kyc-tracking-id="lead-phone-srp" tracking-label="loc=SRP For Sale,prid=43819740">
                            <i class="re__icon-phone-call--sm"></i>
                            <span>0902 765 *** · Hiện số</span>
                        </span>
                        <span class="js__marking-product__KYC re__btn re__btn-se-border--sm re__btn-icon--sm" data-marked-product-info-as-json="{&quot;documentId&quot;:&quot;43819740&quot;,&quot;productId&quot;:43819740,&quot;avatarinfowap&quot;:&quot;https://file4.batdongsan.com.vn/crop/200x140/2025/09/06/20250906104356-ea48_wm.jpg&quot;,&quot;title&quot;:&quot;Bán gấp căn hộ góc 3PN Vinhomes Ba Son, 118.5m2, tầng cao, view sông trực diện, giá tốt nhất 29 tỷ&quot;,&quot;url&quot;:&quot;/ban-can-ho-chung-cu-pho-ton-duc-thang-phuong-ben-nghe-prj-vinhomes-golden-river-ba-son/don-binh-minh-tren-song-sai-gon-moi-ngay-tai-goc-3pn-118m2-dep-nhat-aqua-2-son-pr43819740&quot;}" aria-label="Bấm để lưu tin" data-microtip-position="bottom" role="tooltip" kyc-tracking-id="save-listing" tracking-label="loc=SRP For Sale,status=save" title="" data-change-marking="1">
                            <i class="re__icon-heart--sm"></i>
                            <i class="re__icon-heart-pressed--sm"></i>
                        </span>
                </div>
                <div class="re__clear"></div>
            </div>
        </div>
        <div class="re__clear"></div>
    </a>
    <!-- Because div can contains buttons of links. Show when click they can send the going listing tracking -->
    <!-- So we use anorder way to send going listing tracking -->
    <div class="js__tracking-div-43819740" tracking-id="navigate-srp-ldp" tracking-label="ipos=1,pgno=2,prid=43819740,verified=1,vipType=V0,isMapSearch=0,showMap=0"></div>
    <script type="text/javascript">
        if (!window.CallTrackingWhenGoToLdpPage) {
            window.CallTrackingWhenGoToLdpPage = function (event, container, listingId) {
                var containsTarget = false;
                var trackingQueries = ['[tracking-id]', '[lead-tracking-id]', '.js__entry-professional-agent'];
                for (var i = 0; i < trackingQueries.length; i++) {
                    var query = trackingQueries[i];
                    var elements = container.querySelectorAll(query);
                    for (var j = 0; j < elements.length; j++) {
                        var element = elements[j];
                        if (element.contains(event.target)) {
                            containsTarget = true;
                            break;
                        }
                    }
                    if (containsTarget) {
                        break;
                    }
                }

                if (!containsTarget) {
                    document.querySelector(`.js__tracking-div-${listingId}`).dispatchEvent(new Event('click'))
                }
            };
        }
    </script>
</div>
'''a

In [34]:
cards = driver.find_elements(
    "css selector", "div.js__project-card.js__card-project-web.re__prj-card-full"
)

data = []


for card in cards:
    try:
        # Title
        title = card.find_element(By.CLASS_NAME, "re__prj-card-title").text

        # Location
        location = card.find_element(By.CLASS_NAME, "re__prj-card-location").text

        # Area (first config value)
        area = card.find_elements(By.CLASS_NAME, "re__prj-card-config-value")[0].text

        # Company
        company = card.find_element(By.CLASS_NAME, "re__span-full-max-width").text

        print("Title:", title)
        print("Location:", location)
        print("Area:", area)
        print("Company:", company)
        print("-" * 50)
    except Exception as e:
        print("Error extracting a card:", e)
    data.append({
        "Title": title,
        "Location": location,
        "Area": area,
        "Company" : company,
    })

# Convert to DataFrame


Title: Trellia Cove
Location: Đường Nguyễn Văn Linh, Xã Bình Hưng, Huyện Bình Chánh, Hồ Chí Minh
Area: 8.408 m²
Company: Công ty CP Đầu tư Nam Long
--------------------------------------------------
Error extracting a card: Message: no such element: Unable to locate element: {"method":"css selector","selector":".re__span-full-max-width"}
  (Session info: chrome=140.0.7339.128); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x11dd2a3+66419]
	GetHandleVerifier [0x0x11dd2e4+66484]
	(No symbol) [0x0xfb4bd3]
	(No symbol) [0x0xffe958]
	(No symbol) [0x0xffecfb]
	(No symbol) [0x0xff3f21]
	(No symbol) [0x0x1021064]
	(No symbol) [0x0xff3e24]
	(No symbol) [0x0x10211d4]
	(No symbol) [0x0x10428a1]
	(No symbol) [0x0x1020e16]
	(No symbol) [0x0xff25ce]
	(No symbol) [0x0xff34a4]
	GetHandleVerifier [0x0x1425ee3+2461619]
	GetHandleVerifier [0x0x1420f66+2441270]
	GetHandleVe